In [5]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import albumentations

In [6]:
# 確認 GPU 狀態
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())


CUDA available: True
Number of GPUs: 1


In [7]:
# Step 3: 建立或載入 YOLO 模型
# --------------------------
model = YOLO("yolov11/yolo11l.pt")  # 載入預訓練權重
model.info()


YOLO11l summary: 631 layers, 25,372,160 parameters, 0 gradients, 87.6 GFLOPs


(631, 25372160, 0, 87.6134912)

In [8]:
results = model.train(
    data="yolov9/dataset.yaml",    # 你的資料集設定檔
    epochs=300,
    batch=16,
    lr0=0.001,
    weight_decay=0.002,
    cls=1.2,
    box=9,
    optimizer="SGD",
    plots=True,
    verbose=True,
    

    # # ------------------------------
    # # 以下為調整後的資料增強參數設定
    # # ------------------------------
    
    # # 幾何轉換
    # degrees=5.0,       # 小幅度旋轉，避免破壞結構
    # translate=0.1,     # 平移
    # scale=0.1,         # 縮放
    # shear=0.0,         # 不做剪切
    # perspective=0.0,   # 不做透視
    
    # # 翻轉
    # flipud=0.0,        # 垂直翻轉機率 0%，避免肺部上下顛倒
    # fliplr=0.3,        # 水平翻轉機率 30%，平衡增強與臨床合理性
    
    # # 強度/對比度調整
    # hsv_h=0.0,         # 不做色相調整
    # hsv_s=0.0,         # 不做飽和度調整
    # hsv_v=0.1,         # 亮度微調10%，避免過度改變HU
    
    # # 複合增強
    # mixup=0.1,         # 混合影像機率降低至10%
    # mosaic=0.1,        # 拼接影像機率降低至10%
)


New https://pypi.org/project/ultralytics/8.3.57 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)
engine\trainer: task=detect, mode=train, model=yolov11/yolo11l.pt, data=yolov9/dataset.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

train: Scanning D:\luna\luna_dataset\train.cache... 963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 963/963 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning D:\luna\luna_dataset\val.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]


Plotting labels to runs\detect\train15\labels.jpg... 
optimizer: SGD(lr=0.001, momentum=0.937) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.002), 173 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train15
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    243/300      11.2G     0.6324     0.7112     0.7823          6        640: 100%|██████████| 61/61 [06:21<00:00,  6.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:21<00:00,  5.50s/it]

                   all        118        118      0.892        0.7      0.809       0.45
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 143, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



243 epochs completed in 26.070 hours.
Optimizer stripped from runs\detect\train15\weights\last.pt, 51.2MB
Optimizer stripped from runs\detect\train15\weights\best.pt, 51.2MB

Validating runs\detect\train15\weights\best.pt...
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.69s/it]


                   all        118        118       0.78      0.729      0.794       0.47
Speed: 0.5ms preprocess, 46.4ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs\detect\train15


In [9]:
val_results = model.val()
train_results = model.val(data="yolov9/dataset.yaml", split='train')
test_results = model.val(data="yolov9/dataset.yaml", split='test')

print("Validation Results:", val_results)
print("Train Results:", train_results)
print("Test Results:", test_results)

Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning D:\luna\luna_dataset\val.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:14<00:00,  1.84s/it]


                   all        118        118      0.782      0.729      0.794      0.472
Speed: 0.9ms preprocess, 19.8ms inference, 0.1ms loss, 3.0ms postprocess per image
Results saved to runs\detect\train152
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)


val: Scanning D:\luna\luna_dataset\train.cache... 963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 963/963 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:26<00:00,  2.33it/s]


                   all        963        963      0.988       0.99      0.994      0.852
Speed: 0.6ms preprocess, 17.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\train153
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)


val: Scanning D:\luna\luna_dataset\test.cache... 105 images, 0 backgrounds, 0 corrupt: 100%|██████████| 105/105 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:13<00:00,  1.88s/it]


                   all        105        105      0.808      0.781      0.788      0.471
Speed: 1.7ms preprocess, 19.2ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs\detect\train154
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026A08B32E70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.03